In [4]:
import tensorflow as tf
import pandas as pd

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
print(train)

     SepalLength  SepalWidth  PetalLength  PetalWidth  Species
0            6.4         2.8          5.6         2.2        2
1            5.0         2.3          3.3         1.0        1
2            4.9         2.5          4.5         1.7        2
3            4.9         3.1          1.5         0.1        0
4            5.7         3.8          1.7         0.3        0
..           ...         ...          ...         ...      ...
115          5.5         2.6          4.4         1.2        1
116          5.7         3.0          4.2         1.2        1
117          4.4         2.9          1.4         0.2        0
118          4.8         3.0          1.4         0.1        0
119          5.5         2.4          3.7         1.0        1

[120 rows x 5 columns]


In [39]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
#oddzielenie targets dla modelu
train_y = train.pop('Species')
test_y = test.pop('Species')

In [8]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

# Encode classes as integers
encoder = LabelEncoder()
encoder.fit(train_y)
encoded_Y = encoder.transform(train_y)

# One hot encode integer labels
one_hot_Y = np_utils.to_categorical(encoded_Y)

In [9]:
#normalizacja, poniższa funkcja zwróci nam wartości od 0 do 1, 
#ponadto w wyniku jej procesu utracamy etykiety kolumn i indeksó, więc na nowo są one nadawane
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
 
train_x = pd.DataFrame(scaler.fit_transform(train),
                               columns=train.columns,
                               index=train.index)
test_x = pd.DataFrame(scaler.transform(test),
                           columns=test.columns,
                           index=test.index)

In [10]:
x = train_x.values

In [12]:
y=train_y.values

In [14]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

kf = StratifiedKFold(5, shuffle=True, random_state=42) 

oos_y = []
oos_pred = []
fold = 0

for x_train, x_val in kf.split(x, train_y):
  fold+=1
  print(f"Fold #{fold}")
  xx_train = x[x_train]
  yy_train = y[x_train]
  xx_val = x[x_val]
  yy_val = y[x_val]

  print(xx_train.shape)
  print(yy_train.shape)
  print(yy_train)
  print(xx_val.shape)
  print(yy_val.shape)

  model = Sequential()
  model.add(Dense(10, input_dim=4))
  model.add(BatchNormalization())
  model.add(Dense(3, activation='softmax'))

  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  model.fit(xx_train,yy_train,validation_data=(xx_val,yy_val),verbose=0, epochs=500)
  
  pred = model.predict(xx_val)
    
  oos_y.append(yy_val)
  # raw probabilities to chosen class (highest probability)
  pred = np.argmax(pred,axis=1) 
  oos_pred.append(pred)  

  # Measure this fold's accuracy
  y_compare = np.argmax(yy_val,axis=1) # For accuracy calculation
  score = metrics.accuracy_score(y_compare, pred)
  print(f"Fold score (accuracy): {score}")

  # Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
oos_y_compare = np.argmax(oos_y,axis=1) # For accuracy calculation

score = metrics.accuracy_score(oos_y_compare, oos_pred)
print(f"Final score (accuracy): {score}")    

# Write the cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
#oosDF.to_csv(filename_write,index=False)

Fold #1
(96, 4)
(96,)
[2 1 2 0 1 1 1 0 0 2 2 2 0 2 2 0 2 0 1 2 1 1 1 1 2 2 2 2 2 0 2 2 0 0 2 0 0
 2 0 1 1 0 1 2 2 2 1 2 2 1 2 2 0 0 2 2 0 1 1 1 2 0 1 1 2 0 1 1 0 2 1 0 0 0
 0 2 1 0 0 1 0 1 0 0 0 1 0 2 1 0 2 0 1 0 0 1]
(24, 4)
(24,)


ValueError: ignored

In [43]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

def get_model():
    model = Sequential()
    model.add(Dense(10, input_dim=4))
    model.add(Dense(3, activation='softmax'))

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [44]:
from keras.wrappers.scikit_learn import KerasClassifier

estimator = KerasClassifier(
        build_fn=model,
        epochs=200, batch_size=20,
        verbose=2)

In [48]:
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator, train_x, one_hot_Y, cv=5)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: The first argument to `Layer.call` must always be passed.

  FitFailedWarning)


In [19]:
num_epochs = 20
train_scores = []
test_scores = []
fold_index = 1

# Train model for each fold
for trainSet, testSet in kfold.split(train, one_hot_Y):

    # Create model
    model = get_model()

    # Fit the model
    history = model.fit(
        train[trainSet],
        one_hot_Y[trainSet],
        validation_data=(train[trainSet], one_hot_Y[trainSet]),
        epochs=num_epochs,
        batch_size=5,
        verbose=0
    )

    # Preserve the history and print out some basic stats
    train_scores.append(history.history['acc'])
    test_scores.append(history.history['val_acc'])
    print("Fold %d:" % fold_index)
    print("Training accuracy: %.2f%%" % (history.history['acc'][-1]*100))
    print("Testing accuracy: %.2f%%" % (history.history['val_acc'][-1]*100))

    fold_index += 1

KeyError: ignored